<a href="https://colab.research.google.com/github/kalyani234/CrewAI-FinanceAgent/blob/main/CrewAI_Financial_Management_Multi_Agent_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Project Description

 🤖 Event Management Multi-Agent Framework is an AI-powered system that uses three specialized agents—Venue, Logistics, and Marketing—to plan and manage events.

- The Venue Agent finds suitable venues
- The Logistics Agent confirms bookings and arranges essentials
- the Marketing Agent promotes the event.

Together, they automate event planning from venue selection to final outreach.

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guida

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("open_api")
os.environ["SERPER_API_KEY"] = userdata.get("serpi_api")

# required by CrewAI v0.28
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


In [3]:
from crewai import Agent, Crew, Task

# CrewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Creating Agents

```
 Manager (gpt-4o-mini)
                          |
     ------------------------------------------------
     |                |               |             |
Data Analyst → Strategy Dev → Trade Advisor → Risk Advisor

```

In [17]:
# Agent 1: Data Analyst
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [18]:
 # Agent 2: Trading Strategy Developer
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [19]:
# Agent 3: Trade Advisor
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [20]:
# Agent 4: Risk Advisor
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

# Creating Tasks


```

Data Analyst	Finds market info
Strategy Developer	Creates trading plan
Trade Advisor	Plans when to buy/sell
Risk Advisor	Checks danger and protects

```

In [21]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)


In [22]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [23]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)



In [24]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

# Creating the Crew
The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)
In the example below, it will run this hierarchically.
manager_llm lets you choose the "manager" LLM you want to use.

A hierarchical process in CrewAI means:

*   One main manager AI controls all other agents
*   The manager decides who works first, and who works next
*   Agents do not work randomly — they follow a clear chain of command

It works just like an office manager and employees.

In [25]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-4o-mini",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

Running the Crew
Set the inputs for the execution of the crew.






In [29]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '23000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [30]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to gather insights and alerts regarding significant market opportunities or threats for the selected stock TSLA. This requires continuous monitoring and analysis of market data, which should involve statistical modeling and machine learning techniques to identify trends and predict market movements. 

To achieve this, I will delegate the task to the appropriate co-worker who specializes in data analysis and trading strategy development. The Data Analyst can handle the statistical modeling and data analysis aspects, while the Trading Strategy Developer can focus on the machine learning components. 

First, I will delegate the task of monitoring and analyzing market data to the Data Analyst.

Action: Delegate

# Display the final result as Markdown.

In [31]:
from IPython.display import Markdown
Markdown(result)

### Comprehensive Risk Analysis Report for TSLA Trading Strategies

#### 1. Trend Following Strategy
- **Potential Risks**:
  - **Market Volatility**: Sudden price movements can trigger stop-loss orders, leading to significant losses.
  - **Lagging Indicators**: Moving averages may not respond quickly to market changes, resulting in missed opportunities or losses.
- **Mitigation Strategies**:
  - Implement tighter stop-loss orders to limit potential losses.
  - Utilize additional indicators (e.g., MACD) to confirm trend changes before executing trades.

#### 2. Bollinger Bands Strategy
- **Potential Risks**:
  - **False Signals**: The price may touch the lower band without a genuine reversal, leading to unnecessary buy signals.
  - **Market Sentiment**: In a bearish market, oversold conditions may persist longer than expected.
- **Mitigation Strategies**:
  - Combine Bollinger Bands with volume indicators to confirm trends before entering trades.
  - Set a maximum number of consecutive trades to prevent losses in a volatile environment.

#### 3. RSI Strategy
- **Potential Risks**:
  - **Overbought/Oversold Conditions**: RSI may remain in overbought or oversold territory for extended periods, leading to potential losses.
  - **Market Changes**: Changes in market sentiment can invalidate RSI signals.
- **Mitigation Strategies**:
  - Use additional filters (e.g., moving averages) to validate RSI signals.
  - Regularly review market conditions and adjust RSI thresholds accordingly.

#### 4. Scalping Strategy
- **Potential Risks**:
  - **Execution Risk**: Rapid trades may be affected by slippage, leading to losses.
  - **High Competition**: Scalpers face intense competition, making it challenging to secure favorable prices.
- **Mitigation Strategies**:
  - Utilize high-speed trading platforms to minimize execution delays.
  - Focus on liquid stocks to reduce slippage and ensure rapid execution.

#### 5. Support and Resistance Levels
- **Potential Risks**:
  - **Breakouts and False Breakouts**: Price may break support or resistance levels unexpectedly, leading to potential losses.
  - **Market Sentiment Changes**: Shifts in market sentiment may invalidate established support and resistance levels.
- **Mitigation Strategies**:
  - Confirm breakouts with additional indicators (e.g., volume analysis) before entering trades.
  - Employ trailing stop orders to protect profits in case of reversals.

#### 6. News Sentiment Trading
- **Potential Risks**:
  - **Reaction to News**: Market reactions to news can be unpredictable, leading to volatility and losses.
  - **Delayed Information**: If news is received late, traders may miss optimal entry points.
- **Mitigation Strategies**:
  - Set predefined criteria for news sentiment analysis to guide trading decisions.
  - Use limit orders to manage entry and exit points effectively.

#### 7. Time of Day Considerations
- **Potential Risks**:
  - **Market Behavior Variability**: Different times of the day can yield varying market behaviors, increasing risk during low-volume periods.
  - **Unpredictable Closing Trends**: Selling pressure in the last hour can lead to unexpected losses.
- **Mitigation Strategies**:
  - Monitor pre-market activity closely and adjust strategies based on volume and momentum.
  - Avoid trading during low-volume periods to reduce execution risk.

### Summary
The trading strategies for TSLA present both opportunities and risks, particularly given the current market conditions characterized by significant sales declines and increasing competition. A robust risk management approach, including the implementation of mitigation strategies tailored to each trading strategy, is essential to optimize performance and safeguard against potential losses.